<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="figures/PDSH-cover-small.png">

*このノートブックには、Jake VanderPlas による [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) からの抜粋が含まれています。コンテンツは利用可能です [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do).*

※テキストは[CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode)で、コードは[CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode)で公開しています。このコンテンツが役立つと思われる場合は、[CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode) による作業のサポートを検討してください!*

<!--ナビゲーション-->
< [Working with Time Series](03.11-Working-with-Time-Series.ipynb) | [Working with Time Series](03.11-Working-with-Time-Series.ipynb) | [Working with Time Series](03.11-Working-with-Time-Series.ipynb) >

<a href="https://colab.research.google.com/github/vitroid/PythonDataScienceHandbook/blob/ja/notebooks/03.12-Performance-Eval-and-Query.ipynb"><img align="left" src ="https://colab.research.google.com/assets/colab-badge.svg" alt="Colab で開く" title="Google Colaboratory で開いて実行する"></a>


# 高性能パンダ: eval() および query()

前のセクションで既に見たように、PyData スタックのパワーは、NumPy と Pandas が直感的な構文を介して基本的な操作を C にプッシュする機能に基づいて構築されています。パンダで。
これらの抽象化は、多くの一般的なユース ケースで効率的かつ効果的ですが、多くの場合、一時的な中間オブジェクトの作成に依存しているため、計算時間とメモリ使用量に過度のオーバーヘッドが生じる可能性があります。

バージョン 0.13 (2014 年 1 月リリース) の時点で、Pandas にはいくつかの実験的なツールが含まれており、コストのかかる中間配列の割り当てなしで C 速度の操作に直接アクセスできます。
これらは、[Numexpr](https://github.com/pydata/numexpr) パッケージに依存する ``eval()`` および ``query()`` 関数です。
このノートブックでは、それらの使用方法について説明し、いつそれらを使用することを検討するかについて、いくつかの経験則を示します。

## ``query()`` と ``eval()`` の動機: 複合式

NumPy と Pandas が高速なベクトル化操作をサポートしていることは以前に見ました。たとえば、2 つの配列の要素を追加する場合:

In [1]:
import numpy as np
rng = np.random.RandomState(42)
x = rng.rand(1000000)
y = rng.rand(1000000)
%timeit x + y

100 loops, best of 3: 3.39 ms per loop


[Computation on NumPy Arrays: Universal Functions](02.03-Computation-on-arrays-ufuncs.ipynb) で説明されているように、これは Python ループまたは内包表記を介して追加を行うよりもはるかに高速です。

In [2]:
%timeit np.fromiter((xi + yi for xi, yi in zip(x, y)), dtype=x.dtype, count=len(x))

1 loop, best of 3: 266 ms per loop


ただし、この抽象化は、複合式を計算するときに効率が低下する可能性があります。
たとえば、次の式を考えてみましょう。

In [3]:
mask = (x > 0.5) & (y < 0.5)

NumPy は各部分式を評価するため、これは次のものとほぼ同じです。

In [4]:
tmp1 = (x > 0.5)
tmp2 = (y < 0.5)
mask = tmp1 & tmp2

つまり、*すべての中間ステップが明示的にメモリに割り当てられます*。 ``x`` および ``y`` 配列が非常に大きい場合、メモリと計算のオーバーヘッドが大きくなる可能性があります。
Numexpr ライブラリを使用すると、完全な中間配列を割り当てることなく、要素ごとにこのタイプの複合式を計算することができます。
[Numexpr documentation](https://github.com/pydata/numexpr) には詳細がありますが、当面は、計算したい NumPy スタイルの式を与える *string* をライブラリが受け入れると言うだけで十分です。

In [5]:
import numexpr
mask_numexpr = numexpr.evaluate('(x > 0.5) & (y < 0.5)')
np.allclose(mask, mask_numexpr)

True

ここでの利点は、Numexpr がフルサイズの一時配列を使用しない方法で式を評価するため、特に大きな配列の場合、NumPy よりもはるかに効率的である可能性があることです。
ここで説明する Pandas の ``eval()`` および ``query()`` ツールは概念的に類似しており、Numexpr パッケージに依存しています。

## 効率的な操作のための ``pandas.eval()``

Pandas の ``eval()`` 関数は、文字列式を使用して、``DataFrame`` を使用した操作を効率的に計算します。
たとえば、次の ``DataFrame`` を考えてみましょう:

In [6]:
import pandas as pd
nrows, ncols = 100000, 100
rng = np.random.RandomState(42)
df1, df2, df3, df4 = (pd.DataFrame(rng.rand(nrows, ncols))
                      for i in range(4))

典型的な Pandas のアプローチを使用して 4 つの ``DataFrame`` の合計を計算するには、合計を次のように記述します。

In [7]:
%timeit df1 + df2 + df3 + df4

10 loops, best of 3: 87.1 ms per loop


式を文字列として構築することにより、 pd.eval を介して同じ結果を計算できます。

In [8]:
%timeit pd.eval('df1 + df2 + df3 + df4')

10 loops, best of 3: 42.2 ms per loop


この式の ``eval()`` バージョンは、同じ結果を与えながら、約 50% 高速です (そして、はるかに少ないメモリを使用します):

In [9]:
np.allclose(df1 + df2 + df3 + df4,
            pd.eval('df1 + df2 + df3 + df4'))

True

### ``pd.eval()`` がサポートする操作

Pandas v0.16 の時点で、 pd.eval() は幅広い操作をサポートしています。
これらを実証するために、次の整数 ``DataFrame`` を使用します。

In [10]:
df1, df2, df3, df4, df5 = (pd.DataFrame(rng.randint(0, 1000, (100, 3)))
                           for i in range(5))

#### 算術演算子
``pd.eval()`` はすべての算術演算子をサポートしています。例えば：

In [11]:
result1 = -df1 * df2 / (df3 + df4) - df5
result2 = pd.eval('-df1 * df2 / (df3 + df4) - df5')
np.allclose(result1, result2)

True

#### 比較演算子
``pd.eval()`` は連鎖式を含むすべての比較演算子をサポートしています:

In [12]:
result1 = (df1 < df2) & (df2 <= df3) & (df3 != df4)
result2 = pd.eval('df1 < df2 <= df3 != df4')
np.allclose(result1, result2)

True

#### ビット演算子
``pd.eval()`` は ``&`` および ``|`` ビット演算子をサポートします:

In [13]:
result1 = (df1 < 0.5) & (df2 < 0.5) | (df3 < df4)
result2 = pd.eval('(df1 < 0.5) & (df2 < 0.5) | (df3 < df4)')
np.allclose(result1, result2)

True

さらに、ブール式でリテラル ``and`` および ``or`` の使用をサポートします。

In [14]:
result3 = pd.eval('(df1 < 0.5) and (df2 < 0.5) or (df3 < df4)')
np.allclose(result1, result3)

True

#### オブジェクトの属性とインデックス

pd.eval() は、obj.attr 構文によるオブジェクト属性へのアクセスと、obj[index] 構文によるインデックスへのアクセスをサポートします。

In [15]:
result1 = df2.T[0] + df3.iloc[1]
result2 = pd.eval('df2.T[0] + df3.iloc[1]')
np.allclose(result1, result2)

True

#### その他の操作
関数呼び出し、条件ステートメント、ループ、およびその他のより複雑な構造などの他の操作は、現在 pd.eval() で実装されていません。
これらのより複雑なタイプの式を実行したい場合は、Numexpr ライブラリ自体を使用できます。

## 列単位の操作のための ``DataFrame.eval()``

Pandas にトップレベルの ``pd.eval()`` 関数があるように、``DataFrame`` には同様の方法で動作する ``eval()`` メソッドがあります。
``eval()`` メソッドの利点は、列を *名前で* 参照できることです。
このラベル付き配列を例として使用します。

In [16]:
df = pd.DataFrame(rng.rand(1000, 3), columns=['A', 'B', 'C'])
df.head()

,A,B,C
0,0.375506,0.406939,0.069938
1,0.069087,0.235615,0.154374
2,0.677945,0.433839,0.652324
3,0.264038,0.808055,0.347197
4,0.589161,0.252418,0.557789


上記のように ``pd.eval()`` を使用すると、次のように 3 つの列で式を計算できます。

In [17]:
result1 = (df['A'] + df['B']) / (df['C'] - 1)
result2 = pd.eval("(df.A + df.B) / (df.C - 1)")
np.allclose(result1, result2)

True

``DataFrame.eval()`` メソッドは、列を持つ式のより簡潔な評価を可能にします:

In [18]:
result3 = df.eval('(A + B) / (C - 1)')
np.allclose(result1, result3)

True

ここで、評価された式内で*列名を変数*として扱い、結果が希望どおりになっていることに注意してください。

### DataFrame.eval() での代入

今説明したオプションに加えて、 ``DataFrame.eval()`` は任意の列への代入も可能にします。
以前の ``DataFrame`` を使用してみましょう。これには ``'A'`` 、 ``'B'`` 、および ``'C'`` 列があります:

In [19]:
df.head()

,A,B,C
0,0.375506,0.406939,0.069938
1,0.069087,0.235615,0.154374
2,0.677945,0.433839,0.652324
3,0.264038,0.808055,0.347197
4,0.589161,0.252418,0.557789


``df.eval()`` を使用して新しいカラム ``'D'`` を作成し、それに他のカラムから計算された値を割り当てることができます:

In [20]:
df.eval('D = (A + B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,11.187620
1,0.069087,0.235615,0.154374,1.973796
2,0.677945,0.433839,0.652324,1.704344
3,0.264038,0.808055,0.347197,3.087857
4,0.589161,0.252418,0.557789,1.508776


同様に、既存の列を変更できます。

In [21]:
df.eval('D = (A - B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,-0.449425
1,0.069087,0.235615,0.154374,-1.078728
2,0.677945,0.433839,0.652324,0.374209
3,0.264038,0.808055,0.347197,-1.566886
4,0.589161,0.252418,0.557789,0.603708


### DataFrame.eval() のローカル変数

``DataFrame.eval()`` メソッドは、ローカルの Python 変数を操作できる追加の構文をサポートしています。
次の点を考慮してください。

In [22]:
column_mean = df.mean(1)
result1 = df['A'] + column_mean
result2 = df.eval('A + @column_mean')
np.allclose(result1, result2)

True

ここでの ``@`` 文字は *列名* ではなく *変数名* をマークし、列の名前空間と Python オブジェクトの名前空間の 2 つの「名前空間」を含む式を効率的に評価できるようにします。
この ``@`` 文字は ``DataFrame.eval()`` *メソッド* でのみサポートされており、``pandas.eval()`` *関数* ではサポートされていないことに注意してください。 ()`` 関数は、1 つの (Python) 名前空間にのみアクセスできます。

## DataFrame.query() メソッド

``DataFrame`` には、``query()`` メソッドと呼ばれる、評価された文字列に基づく別のメソッドがあります。
次の点を考慮してください。

In [23]:
result1 = df[(df.A < 0.5) & (df.B < 0.5)]
result2 = pd.eval('df[(df.A < 0.5) & (df.B < 0.5)]')
np.allclose(result1, result2)

True

``DataFrame.eval()`` の説明で使用した例と同様に、これは ``DataFrame`` の列を含む式です。
ただし、 ``DataFrame.eval()`` 構文を使用して表現することはできません!
代わりに、このタイプのフィルタリング操作では、query() メソッドを使用できます。

In [24]:
result2 = df.query('A < 0.5 and B < 0.5')
np.allclose(result1, result2)

True

より効率的な計算であることに加えて、マスキング式と比較して、これは読みやすく理解しやすいものです。
``query()`` メソッドは、ローカル変数をマークする ``@`` フラグも受け入れることに注意してください:

In [25]:
Cmean = df['C'].mean()
result1 = df[(df.A < Cmean) & (df.B < Cmean)]
result2 = df.query('A < @Cmean and B < @Cmean')
np.allclose(result1, result2)

True

## パフォーマンス: これらの関数をいつ使用するか

これらの関数を使用するかどうかを検討する場合、*計算時間*と*メモリ使用量*の 2 つの考慮事項があります。
メモリ使用量は、最も予測可能な側面です。すでに述べたように、NumPy 配列または Pandas ``DataFrame`` を含むすべての複合式は、一時的な配列を暗黙的に作成します。
たとえば、次のようになります。

In [26]:
x = df[(df.A < 0.5) & (df.B < 0.5)]

これとほぼ同等です：

In [27]:
tmp1 = df.A < 0.5
tmp2 = df.B < 0.5
tmp3 = tmp1 & tmp2
x = df[tmp3]

一時的な ``DataFrame`` のサイズが利用可能なシステム メモリ (通常は数ギガバイト) に比べて大きい場合は、 ``eval()`` または ``query()`` 式を使用することをお勧めします。
これを使用して、配列のおおよそのサイズをバイト単位で確認できます。

In [28]:
df.values.nbytes

32000

パフォーマンスの面では、システムメモリを使い果たしていなくても eval() の方が高速です。
問題は、一時的な ``DataFrame`` が、システムの L1 または L2 CPU キャッシュのサイズ (通常、2016 年には数メガバイト) とどのように比較されるかです。それらがはるかに大きい場合、 ``eval()`` は、異なるメモリ キャッシュ間での潜在的に遅い値の移動を回避できます。
実際には、従来の方法と ``eval``/``query`` メソッドとの間の計算時間の差は、通常はそれほど重要ではないことがわかりました。どちらかといえば、従来の方法の方が小さい配列では高速です!
``eval``/``query`` の利点は、主にメモリの節約と、それらが提供するよりきれいな構文にあります。

``eval()`` と ``query()`` の詳細のほとんどをここでカバーしました。これらの詳細については、Pandas のドキュメントを参照してください。
特に、これらのクエリを実行するために、さまざまなパーサーとエンジンを指定できます。詳細については、["Enhancing Performance" section](http://pandas.pydata.org/pandas-docs/dev/enhancingperf.html) 内の説明を参照してください。

<!--ナビゲーション-->
< [Working with Time Series](03.11-Working-with-Time-Series.ipynb) | [Working with Time Series](03.11-Working-with-Time-Series.ipynb) | [Working with Time Series](03.11-Working-with-Time-Series.ipynb) >

<a href="https://colab.research.google.com/github/vitroid/PythonDataScienceHandbook/blob/ja/notebooks/03.12-Performance-Eval-and-Query.ipynb"><img align="left" src ="https://colab.research.google.com/assets/colab-badge.svg" alt="Colab で開く" title="Google Colaboratory で開いて実行する"></a>
